In [1]:
#imports
import numpy as np
from at_synapse_detection import processDetections as pd
from at_synapse_detection import SynapseDetection as syn
from at_synapse_detection import dataAccess as da
from at_synapse_detection import synaptogram
import importlib
import matplotlib.pyplot as plt
from skimage import measure
import scipy.ndimage as ndimage
from shapely import geometry
import math
import scipy

In [22]:
importlib.reload(pd)
importlib.reload(syn)
importlib.reload(da)

<module 'at_synapse_detection.dataAccess' from '/Users/anish/Documents/Connectome/SynapseAnalysis/at_synapse_detection/dataAccess.py'>

In [3]:
args = {
    "EM_annotation_json":"../data/M247514_Rorb_1/Site3Align2/json_annotations/m247514_Site3Annotation_MN_global_v2.json",
    "LM_annotation_json":"../data/M247514_Rorb_1/Site3Align2/results/resultVol9.json",
    "EM_metadata_csv":"../data/M247514_Rorb_1/Site3Align2/MNSite3Synaptograms_v2.csv",
    "LM_metadata_file":"../data/M247514_Rorb_1/Site3Align2/site3_metadata.json",
    "EM_inclass_column":"glutsynapse",
    "EM_not_synapse_column":"ConsensusNotSynapse",
    "output_json":"../data/M247514_Rorb_1/Site3Align2/results/Anish_evaluation_output.json", 
    "annotationToRemove": "../data/M247514_Rorb_1/Site3Align2/missedanno.json"
    }

In [4]:
metadataFN = '../data/M247514_Rorb_1/Site3Align2/site3_metadata.json'
metadata = syn.loadMetadata(metadataFN)
queryFN = metadata['querylocation']
listOfQueries = syn.loadQueriesJSON(queryFN)

In [5]:
len(listOfQueries)

8

In [68]:
listOfQueryNumbers = [  0,   1,   2,   3,   4,   5,   6,   7]
listOfThresholds   = [0.8, 0.7, 0.7, 0.8, 0.7, 0.7, 0.8, 0.8]
queryresult = pd.combineResultVolumes(listOfQueryNumbers, listOfThresholds, metadata, args)

# listOfQueryNumbers = [  0]
# listOfThresholds   = [0.7]
# queryresult = pd.combineResultVolumes(listOfQueryNumbers, listOfThresholds, metadata, args)

/Users/anish/Documents/Connectome/Synaptome-Duke/data/collman17/Site3Align2Stacks/resultVol0.npy
/Users/anish/Documents/Connectome/Synaptome-Duke/data/collman17/Site3Align2Stacks/resultVol1.npy
/Users/anish/Documents/Connectome/Synaptome-Duke/data/collman17/Site3Align2Stacks/resultVol2.npy
/Users/anish/Documents/Connectome/Synaptome-Duke/data/collman17/Site3Align2Stacks/resultVol3.npy
/Users/anish/Documents/Connectome/Synaptome-Duke/data/collman17/Site3Align2Stacks/resultVol4.npy
/Users/anish/Documents/Connectome/Synaptome-Duke/data/collman17/Site3Align2Stacks/resultVol5.npy
/Users/anish/Documents/Connectome/Synaptome-Duke/data/collman17/Site3Align2Stacks/resultVol6.npy
/Users/anish/Documents/Connectome/Synaptome-Duke/data/collman17/Site3Align2Stacks/resultVol7.npy
156
156
EM_per_LM [ 0.45839416  0.46569343  0.07591241]
LM_per_EM [ 0.22720247  0.76043277  0.01236476]
lm edge detections: 9603
em edge annotations 156
LM detections: 10299


In [7]:
good_annotations = queryresult['good_annotations']

In [30]:
EM_edge = queryresult['EM_edge']

In [33]:
sum(EM_edge)

156

In [53]:
anno = good_annotations[32]
print(anno['oid'])
win_xy = 13
win_z = 2
filepath = '/Users/anish/Documents/Connectome/Synaptome-Duke/data/collman17/Site3Align2Stacks/'
query = listOfQueries[0]
threshlist = [0.7, 0.8, 0.9]


635


In [54]:
output = np.zeros((len(good_annotations), len(listOfQueries), len(threshlist)))

In [55]:
synapseDetected = syn.checkQueryAgainstAnno(anno, query, threshlist, win_xy, win_z, filepath)

In [56]:
synapseDetected

array([ True, False, False], dtype=bool)

In [57]:
for synInd, anno in enumerate(good_annotations): 
    print(anno['oid'])
    
    for queryInd, query in enumerate(listOfQueries): 
        synapseDetected = syn.checkQueryAgainstAnno(anno, query, threshlist, win_xy, win_z, filepath)    
        output[synInd, queryInd, :] = synapseDetected

559
561
563
568
570
574
576
579
581
583
585
587
589
591
593
595
597
601
603
605
607
609
611
613
615
619
621
623
627
629
631
633
635
637
639
641
643
645
647
649
651
655
657
659
661
663
665
667
669
671
673
675
679
681
683
687
689
691
693
697
699
701
705
707
709
711
715
717
719
721
723
725
729
731
733
735
737
739
743
745
747
749
751
753
755
759
761
763
765
769
771
773
775
777
781
783
785
789
791
793
797
799
801
803
807
809
811
817
819
821
823
825
827
829
831
835
837
839
841
845
849
851
853
855
857
859
861
863
865
867
869
871
873
877
879
881
883
885
887
889
895
897
901
903
905
907
909
913
915
917
919
921
923
929
931
933
935
937
939
941
943
945
947
949
951
953
955
957
959
961
963
967
971
973
975
977
979
981
983
985
987
989
993
995
997
999
1001
1003
1005
1007
1009
1011
1013
1015
1017
1019
1021
1025
1027
1029
1031
1033
1037
1041
1043
1045
1047
1049
1051
1055
1057
1059
1061
1063
1065
1067
1069
1071
1073
1075
1077
1079
1083
1085
1087
1089
1091
1093
1095
1097
1099
1105
1107
1109
1111
1113
1115
1

In [62]:
foo = output[:, 0, 0]

In [63]:
querycount = np.zeros((len(listOfQueries), len(threshlist), 1))

In [64]:
for q_n in range(0, output.shape[1]): 
    
    for t_n in range(0, output.shape[2]):
        
        foo = output[:, q_n, t_n]
        querycount[q_n, t_n, 0] = np.sum(foo) / len(foo)
        

In [65]:
querycount

array([[[ 0.9003736 ],
        [ 0.82316314],
        [ 0.7135741 ]],

       [[ 0.79950187],
        [ 0.69987547],
        [ 0.55043587]],

       [[ 0.7758406 ],
        [ 0.64009963],
        [ 0.51681196]],

       [[ 0.92154421],
        [ 0.84308842],
        [ 0.73972603]],

       [[ 0.80821918],
        [ 0.67496887],
        [ 0.53798257]],

       [[ 0.64383562],
        [ 0.51058531],
        [ 0.35242839]],

       [[ 0.67870486],
        [ 0.5267746 ],
        [ 0.29389788]],

       [[ 0.67870486],
        [ 0.5267746 ],
        [ 0.29389788]]])

In [66]:
listOfQueries

[{'postIF': ['PSD95'],
  'postIF_z': [2],
  'preIF': ['synapsin'],
  'preIF_z': [1],
  'thresh': 0.8},
 {'postIF': ['PSD95'],
  'postIF_z': [2],
  'preIF': ['synapsin'],
  'preIF_z': [2],
  'thresh': 0.7},
 {'postIF': ['PSD95'],
  'postIF_z': [3],
  'preIF': ['synapsin'],
  'preIF_z': [1],
  'thresh': 0.7},
 {'postIF': ['PSD95'],
  'postIF_z': [2],
  'preIF': ['VGlut1'],
  'preIF_z': [1],
  'thresh': 0.8},
 {'postIF': ['PSD95'],
  'postIF_z': [3],
  'preIF': ['VGlut1'],
  'preIF_z': [1],
  'thresh': 0.7},
 {'postIF': ['PSD95'],
  'postIF_z': [2],
  'preIF': ['synapsin', 'VGlut1'],
  'preIF_z': [1, 1],
  'thresh': 0.7},
 {'postIF': ['PSD95', 'GluN1'],
  'postIF_z': [2, 1],
  'preIF': ['VGlut1'],
  'preIF_z': [1],
  'thresh': 0.9},
 {'postIF': ['PSD95', 'GluN1'],
  'postIF_z': [2, 1],
  'preIF': ['synapsin'],
  'preIF_z': [1],
  'thresh': 0.9}]

In [75]:
oidlist = []
missedList = [] 

for n, anno in enumerate(good_annotations): 
    oidlist.append(int(anno['oid']))
    foo = output[n, :, :]
    if np.sum(foo) ==0: 
        missedList.append(int(anno['oid']))

In [76]:
len(missedList)

48

In [72]:
oid = oidlist.index(635)
output[oid, :, :]


array([[ 1.,  0.,  0.],
       [ 0.,  0.,  0.],
       [ 0.,  0.,  0.],
       [ 1.,  0.,  0.],
       [ 0.,  0.,  0.],
       [ 0.,  0.,  0.],
       [ 0.,  0.,  0.],
       [ 0.,  0.,  0.]])

In [69]:
missed_annotations = queryresult['missed_annotations']
missed_anno_ids = pd.getMissedAnnoIds(missed_annotations)

In [70]:
len(missed_anno_ids)

147

In [87]:
missed_anno_ids_int = [] 

for id in missed_anno_ids: 
    missed_anno_ids_int.append(int(id))

In [83]:
missedList

[593,
 603,
 651,
 755,
 771,
 811,
 827,
 897,
 909,
 921,
 1063,
 1071,
 1139,
 1345,
 1369,
 1391,
 1393,
 1409,
 1415,
 1519,
 1571,
 1591,
 1641,
 1679,
 1693,
 1701,
 1743,
 1747,
 1763,
 1799,
 1801,
 1803,
 1873,
 1879,
 1883,
 1917,
 1953,
 1959,
 1975,
 2095,
 2262,
 2300,
 2390,
 2488,
 2500,
 2586,
 2758,
 2822]

In [91]:
differencelist = [x for x in (missed_anno_ids_int) if x not in (missedList)]

In [92]:
differencelist

[581,
 607,
 623,
 631,
 635,
 649,
 673,
 691,
 729,
 783,
 791,
 819,
 857,
 863,
 901,
 905,
 923,
 931,
 949,
 955,
 963,
 971,
 987,
 995,
 1001,
 1003,
 1009,
 1069,
 1085,
 1117,
 1167,
 1189,
 1201,
 1217,
 1223,
 1227,
 1269,
 1273,
 1283,
 1299,
 1367,
 1371,
 1435,
 1443,
 1453,
 1457,
 1463,
 1471,
 1483,
 1513,
 1535,
 1553,
 1569,
 1577,
 1587,
 1593,
 1613,
 1625,
 1631,
 1737,
 1739,
 1755,
 1791,
 1811,
 1835,
 1847,
 1881,
 1885,
 1891,
 1903,
 1909,
 1919,
 1921,
 1941,
 1949,
 1957,
 1961,
 1963,
 1985,
 2005,
 2033,
 2047,
 2055,
 2091,
 2105,
 2139,
 2182,
 2186,
 2216,
 2226,
 2282,
 2326,
 2338,
 2360,
 2404,
 2406,
 2420,
 2434,
 2442,
 2474,
 2478,
 2532,
 2534,
 2562,
 2582,
 2720,
 2724,
 2754,
 2802,
 2810,
 2812]